In [ ]:
import numpy as np
import healpy as hp
from pathlib import Path

In [ ]:
output_dir = Path("production-data") / "dust_gnilc"

In [ ]:
datadir = output_dir / "raw"

In [ ]:
output_nside = 2048
name = "beta"

In [ ]:
output_lmax = int(min(2.5 * output_nside, 8192 * 2))

## Galactic mask

In [ ]:
galactic_mask = (
    hp.ud_grade(
        hp.read_map(datadir / "HFI_Mask_GalPlane-apo2_2048_R2.00_GAL080_noapo.fits.gz"),
        output_nside,
    )
    == 1
)

## Large scales

In [ ]:
alm_large_scale = hp.read_alm(
    datadir / f"gnilc_dust_largescale_template_{name}_alm_nside2048_lmax1024_complex64.fits.gz",
    hdu=1,
)

In [ ]:
map_large_scale = hp.alm2map(alm_large_scale.astype(np.complex128), nside=output_nside)

## Small scales modulation

In [ ]:
modulate_alm = hp.read_alm(
    datadir / f"gnilc_dust_temperature_modulation_alms_lmax3072.fits.gz"
).astype(np.complex128)

## Small scales

In [ ]:
cl_small_scale = hp.read_cl(
    datadir / f"gnilc_dust_small_scales_{name}_cl_lmax16384.fits.gz"
)

In [ ]:
synalm_lmax = 8192 * 2  # for reproducibility
# synalm_lmax = output_lmax
seed = 777 if name == "beta" else 888
np.random.seed(seed)

alm_small_scale = hp.synalm(
    cl_small_scale,
    lmax=synalm_lmax,
    new=True,
)

alm_small_scale = hp.almxfl(alm_small_scale, np.ones(int(2.5 * output_nside)))
map_small_scale = hp.alm2map(alm_small_scale, nside=output_nside)
assert np.isnan(map_small_scale).sum() == 0

In [ ]:
map_small_scale = hp.alm2map(alm_small_scale, nside=output_nside)

In [ ]:
hp.mollview(map_small_scale)

In [ ]:
map_small_scale[galactic_mask] *= hp.alm2map(modulate_alm, output_nside)[galactic_mask]

In [ ]:
hp.mollview(map_small_scale)

In [ ]:
hp.mollview(galactic_mask)

## Combine scales

* Combine small and large scale maps
* Transform from logpoltens to IQU
* Write output map

In [ ]:
map_total = map_large_scale + map_small_scale

In [ ]:
hp.write_map(
    output_dir / f"gnilc_dust_{name}_nside{output_nside}.fits",
    map_total,
    coord="G",
    column_units="" if name == "beta" else "K",
    extra_header = [("lmax", output_lmax), ("ref_freq", "353 GHz")],
    dtype=np.float32,
    overwrite=True,
)

In [ ]:
hp.mollview(map_large_scale, title="Large scale")
hp.mollview(map_small_scale, title="Small scale")
hp.mollview(map_total, title="Total")